In [122]:
import time
import numpy as np
import pandas as pd
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import matplotlib.pyplot as plt
from scipy.spatial import distance


In [123]:
def load_data(file_path, d=2, norm=True):
    objects = pd.read_pickle(file_path)
    pos_images, neg_images = [], []    
    for image, label in zip(objects['data'], objects['labels']):        
        if label == d%5 or label == (d+1)%5:
            image = image.flatten()
            if norm:
                image = image/255.0
            if label == d%5:                
                pos_images.append(image)                    
            else:
                neg_images.append(image)
    pos_images = np.array(pos_images)
    neg_images = np.array(neg_images)
    pos_labels = np.array([1 for _ in range(len(pos_images))])
    neg_labels = np.array([-1 for _ in range(len(neg_images))])
    labels = np.concatenate((pos_labels, neg_labels))
    images = np.vstack((pos_images, neg_images))
    return images, np.reshape(labels, (len(labels), 1))


In [124]:
def solve(images, labels, kernel=0, gamma=0.001):
    def get_cvxopt_solver_parameters(X, y, C=1.0):
        m, n = X.shape
        y = y.reshape(len(y),1) * 1.0        
        if kernel == 0:              
            X_dash = X*y
            H = np.dot(X_dash, X_dash.T) * 1.0
        else:                            
            pairwise_distance = distance.cdist(X, X, 'sqeuclidean')                        
            H = np.exp(-gamma*pairwise_distance)*(y @ y.T)             
        P = cvxopt_matrix(H)
        q = cvxopt_matrix(-np.ones((m, 1)))
        G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
        A = cvxopt_matrix(y.T)
        b = cvxopt_matrix(np.zeros(1))
        return P, q, G, h, A, b, H
    P, q, G, h, A, b, H = get_cvxopt_solver_parameters(images, labels)
    solution = cvxopt_solvers.qp(P, q, G, h, A, b)    
    return solution, H

In [125]:
X, y = load_data("part2_data/part2_data/train_data.pickle")          
print(X)

[[0.64313725 0.80784314 0.32941176 ... 0.47843137 0.66666667 0.17254902]
 [0.06666667 0.01176471 0.00784314 ... 0.1254902  0.09019608 0.0627451 ]
 [0.77254902 0.73333333 0.7372549  ... 0.30980392 0.25490196 0.22352941]
 ...
 [1.         1.         1.         ... 0.99215686 0.99215686 0.99215686]
 [0.41960784 0.30588235 0.2627451  ... 0.49411765 0.4        0.38039216]
 [0.70588235 0.70588235 0.70588235 ... 0.73333333 0.73333333 0.73333333]]


In [126]:
solution, _ = solve(X, y, kernel=1)

     pcost       dcost       gap    pres   dres
 0: -2.5416e+03 -1.3415e+04  7e+04  4e+00  5e-13
 1: -1.7151e+03 -1.0205e+04  1e+04  3e-01  4e-13
 2: -1.8445e+03 -2.9955e+03  1e+03  1e-02  4e-13
 3: -2.2350e+03 -2.5476e+03  3e+02  3e-03  4e-13
 4: -2.3437e+03 -2.4451e+03  1e+02  7e-04  4e-13
 5: -2.3773e+03 -2.4095e+03  3e+01  2e-04  4e-13
 6: -2.3889e+03 -2.3972e+03  8e+00  4e-05  5e-13
 7: -2.3924e+03 -2.3934e+03  1e+00  4e-06  5e-13
 8: -2.3929e+03 -2.3929e+03  5e-02  2e-07  5e-13
 9: -2.3929e+03 -2.3929e+03  2e-03  5e-09  5e-13
Optimal solution found.


In [142]:
alphas = np.ravel(solution['x'])
#np.savetxt("alphas.csv", alphas, delimiter=',')

print(max(alphas))
print(min(alphas))

0.9999999644938747
3.609954844414134e-08


In [141]:
threshold = 0
n_samples, n_features = X.shape    
n_support_ = 0
support_vectors = []
for i in range(n_samples):        
    if alphas[i] > threshold:
        n_support_ += 1
        support_vectors.append(i)
print(n_support_, n_support_/n_samples * 100)

4000 100.0


In [132]:
pairwise_distance = distance.cdist(X, X, 'sqeuclidean')
K = np.exp(-gamma*pairwise_distance)
print(K)

[[1.         0.58374799 0.73803472 ... 0.63624261 0.81742163 0.69480429]
 [0.58374799 1.         0.53120504 ... 0.48106599 0.70951476 0.45851254]
 [0.73803472 0.53120504 1.         ... 0.63990452 0.79477925 0.74063605]
 ...
 [0.63624261 0.48106599 0.63990452 ... 1.         0.67141916 0.59121143]
 [0.81742163 0.70951476 0.79477925 ... 0.67141916 1.         0.72697659]
 [0.69480429 0.45851254 0.74063605 ... 0.59121143 0.72697659 1.        ]]


In [133]:
b =  0.0
maxterm, minterm  = float('-inf'), float('inf') 
def compute(j):
    sum = 0.0
    for i in range(n_support_):
        sum += alphas[support_vectors[i]]*y[support_vectors[i]]*K[support_vectors[i]][j]        
    return sum 

for j in range(n_samples):           
    if y[j] == -1:
        maxterm = max(maxterm, compute(j))                 
    else:
        minterm = min(minterm, compute(j))        
b = (-1/2) * (maxterm + minterm)
print(b)

[-5.40706815]


In [134]:
def predict(x, b):
    return 1 if x+b > 0 else -1
def accuracy(y, b):#train accuracy
    correct = 0
    for i in range(n_samples):
        if y[i] == predict(w_t_x[i], b):
            correct += 1
    return correct / n_samples

# Testing

In [135]:
X_test, y_test = load_data("part2_data/part2_data/test_data.pickle")    
n_samples_test, n_features = X_test.shape

In [136]:
gamma = 0.001
K_test = np.zeros((n_support_, n_samples_test))
for i in range(n_support_):
    for j in range(n_samples_test):
        K_test[i][j] = -gamma*np.linalg.norm(X[support_vectors[i]] - X_test[j])**2
print(K_test)

[[-0.53315185 -0.43264258 -0.23947922 ... -0.49393055 -0.48178716
  -0.33931755]
 [-0.25040551 -0.52396271 -0.35085172 ... -0.69782467 -0.40005833
  -0.37583353]
 [-0.20592172 -0.26345466 -0.17549086 ... -0.50003256 -0.28127137
  -0.29691329]
 ...
 [-0.34144301 -0.67575728 -0.48079912 ... -0.75484603 -0.50881201
  -0.41289367]
 [-0.19138923 -0.23538165 -0.14952314 ... -0.33443734 -0.19949097
  -0.18734647]
 [-0.39868829 -0.59008088 -0.54390648 ... -0.53678175 -0.52664278
  -0.40199316]]


In [137]:
def prediction(j):
    wTx = 0.0
    for i in range(n_support_):
        wTx += alphas[support_vectors[i]]*y[support_vectors[i]]*K_test[i][j]       
    return 1 if wTx + b[0] > 0 else -1

In [138]:
def test_accuracy():
    correct = 0.0
    for i in range(n_samples_test):                
        if y_test[i] == prediction(i):
            correct += 1
    return correct/n_samples_test
print(test_accuracy())

0.5865


In [ ]:
def get_images(k=5):
    